In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
!unzip /content/drive/My\ Drive/animal_classification/train.zip -d /tmp

Archive:  /content/drive/My Drive/animal_classification/train.zip
replace /tmp/train/elephants/ASG0016tov_1.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/drive/My\ Drive/animal_classification/validation.zip -d /tmp

Archive:  /content/drive/My Drive/animal_classification/validation.zip
replace /tmp/validation/elephants/ASG0018o5u_0.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip /content/drive/My\ Drive/animal_classification/test.zip -d /tmp

In [ ]:
!ls /tmp/train/elephants|wc -l
!ls /tmp/validation/elephants|wc -l
!ls /tmp/test/test |wc -l

In [ ]:
train_dir = '/tmp/train'
validation_dir = '/tmp/validation'
test_dir = '/tmp/test'

In [ ]:
batch_size = 32
IMG_HEIGHT = 330
IMG_WIDTH = 330

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [ ]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(os.path.join(train_dir, 'elephants/ASG001c0fg_1.jpeg'))
imgplot = plt.imshow(img)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:5])

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D (pool_size=(2, 2), strides=(1,1), padding='valid', data_format=None),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='valid', data_format=None),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(1,1), padding='valid', data_format=None),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid', data_format=None),           
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
epochs = 20
checkpoint_path = '/tmp/best_model.ckpt'
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, monitor='val_loss', save_weights_only=True, 
    save_best_only=True, mode='min', verbose=1)
history = model.fit_generator(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen,
    callbacks=[cp_callback]
)

In [ ]:
train_data_gen.reset()
predictions = model.predict_generator(train_data_gen, verbose=1)

In [ ]:
predictions.shape

In [ ]:
for i in train_data_gen:

    idx = (train_data_gen.batch_index - 1) * train_data_gen.batch_size

    print(train_data_gen.filenames[idx : idx + train_data_gen.batch_size])
    break

In [ ]:
train_data_gen.filenames[-1:]

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.load_weights(checkpoint_path)
loss = model.evaluate_generator(val_data_gen, verbose=2)

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our test data
test_data_gen = test_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=test_dir,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode=None,
                                                           shuffle=False
                                                         )
test_data_gen.reset()
predictions = model.predict_generator(test_data_gen, verbose=1)

In [ ]:
test_data_gen.filenames = list(map(lambda x: os.path.basename(x), test_data_gen.filenames))
results = pd.DataFrame({"id": test_data_gen.filenames, "label": predictions[:, 0]})
results.to_csv("results13.csv",index=False)